In [14]:
import numpy as np
import cv2
import time
import pandas as pd 

In [15]:
cap = cv2.VideoCapture("../videos/DJI_0035.MOV")

suc, prev = cap.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
h, w = list(prevgray.shape[:2])[::-1]
prevgray = cv2.resize(prevgray,( h // 4, w // 4))

In [16]:
prevgray.shape

(540, 960)

In [17]:
flow_list = []
count = 0
while True:
    count += 1

    suc, img = cap.read()
    if (suc ):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray, (h // 4, w // 4))
        # start time to calculate FPS
        start = time.time()


        flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        prevgray = gray
        if count % 20 == 0:
            flow_list.append(flow.tolist())
            # print(count)
    else:
        break
    

In [18]:
cap.release()

In [19]:
len(flow_list)

8

In [20]:
speeds = np.array(flow_list)
speeds.shape 

(8, 540, 960, 2)

In [21]:
arr = None
with open('./speed_per_20.npz', 'rb') as f:
    arr = np.load(f)

In [22]:
(arr == speeds).sum()

1694372

In [23]:
with open('./speed_per_20(1).npz', 'wb') as f:
    np.save(f, speeds)

In [24]:
calc_speed = lambda x : np.sqrt(x[::, ::, 0] ** 2 + x[::, ::, 1] ** 2)

In [25]:
speed_list = map(calc_speed, [speeds[i] for i in range(speeds.shape[0])])

In [26]:
speed_list = [*speed_list]

In [27]:
real_speed = np.array("1.013	2.867	4.263	4.442	4.512	4.314	4.658	4.229	3.516	3.814	2.278".split("\t"), dtype = "float")

In [28]:
aixs_line = 0

In [29]:
speed_list[0]
# hsv = np.zeros((h, w, 3), np.uint8)
# hsv[...,0] = ang*(180/np.pi/2)
# hsv[...,1] = 255
# hsv[...,2] = np.minimum(v*4, 255)

array([[3.06632756e-02, 5.98023080e-02, 7.80131999e-02, ...,
        1.08249308e-01, 9.46387472e-02, 6.55386718e-02],
       [3.94036578e-02, 5.55386020e-02, 6.12093519e-02, ...,
        9.63328421e-02, 7.93351332e-02, 5.21984973e-02],
       [3.97662915e-02, 5.10036467e-02, 5.84997853e-02, ...,
        8.08399753e-02, 6.58388553e-02, 4.45738083e-02],
       ...,
       [2.23374025e-02, 3.11830848e-02, 3.74330176e-02, ...,
        1.89893988e+01, 1.88479363e+01, 1.82025862e+01],
       [2.24912128e-02, 3.28106970e-02, 5.02964506e-02, ...,
        1.85694209e+01, 1.83405585e+01, 1.75837132e+01],
       [1.78305310e-02, 2.74647693e-02, 4.41655852e-02, ...,
        1.75983312e+01, 1.70856661e+01, 1.60721290e+01]])

In [34]:
angs = np.arctan2(speeds[:, :, :, 0], speeds[:, :, :, 1] ) + np.pi